In [ ]:
from PIL import Image, ImageOps
import numpy as np
import os
import holoviews as hv; hv.extension('bokeh', logo=False)

hv.opts.defaults(hv.opts.Image(cmap="gray",invert_yaxis=True, xaxis=None, yaxis=None, data_aspect=1))

In [ ]:
def image_axes(im):
    inch_mm          = 25.4

    dpi              = im.info['dpi']
    xmax_mm, ymax_mm = im.width/dpi[0]*inch_mm,im.height/dpi[1]*inch_mm
    xs               = np.linspace(0,xmax_mm,im.width)
    ys               = np.linspace(0,ymax_mm,im.height)
    return xs, ys

img  = Image.open( os.path.join('..','DATA','dog.jpg') )
A    = np.array(img)
B    = np.array(ImageOps.grayscale(img)) # Convert RGB to grayscale

img_x, img_y = image_axes( img )

In [ ]:
## Denoise
Bnoise  = B + 200*np.random.randn(*B.shape).astype('uint8') # Add some noise
Bt      = np.fft.fft2(Bnoise)
Btshift = np.fft.fftshift(Bt)
F       = np.log(np.abs(Btshift)+1)    # Put FFT on log scale

In [ ]:
nx,ny = B.shape
X,Y   = np.meshgrid(np.arange(-ny/2+1,ny/2+1),np.arange(-nx/2+1,nx/2+1))
# xgrid = np.fft.ifftshift(np.arange(-nx/2+1,nx/2+1))
# ygrid = np.fft.ifftshift(np.arange(-ny/2+1,ny/2+1))
# X,Y = np.meshgrid(ygrid,xgrid)
R2          = np.power(X,2) + np.power(Y,2)
ind         = R2 < 150**2
Btshiftfilt = Btshift * ind
Ffilt       = np.log(np.abs(Btshiftfilt)+1) # Put FFT on log scale

Btfilt = np.fft.ifftshift(Btshiftfilt)
Bfilt = np.fft.ifft2(Btfilt).real

In [ ]:
h=\
hv.Image((img_x,img_y, Bnoise ), kdims=['x','y'], vdims="data").opts(title='Noisy Image')+\
hv.Image((img_x,img_y, F      ), kdims=['x','y'], vdims="data").opts(title='2D FFT')+\
hv.Image((img_x,img_y, Bfilt ), kdims=['x','y'], vdims="data").opts(title='Denoised Image')+\
hv.Image((img_x,img_y, Ffilt ), kdims=['x','y'], vdims="data").opts(title='Filtered 2D FFT')

h.opts(title='Denoised Images', shared_axes=False).cols(2)